For the crawl, the 4pm london fix each day. 

(site)[https://www.bloomberg.com/markets/currencies/fx-fixings]

In [59]:
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install BeautifulSoup4
!{sys.executable} -m pip install pandas

    100% |████████████████████████████████| 15.9MB 784kB/s ta 0:00:011
    100% |████████████████████████████████| 13.9MB 1.5MB/s ta 0:00:011
    100% |████████████████████████████████| 512kB 4.3MB/s ta 0:00:01


In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from random import randint, uniform
from time import sleep
import pandas as pd

### Extract data from a page 
**Note to self**: in the future, data structure should not be organized by day/dict but cp/dict, with each dict containing data akin to a time series.

In [2]:
def extract_currency_pairs(bs):
    cp = []
    cprows = bs.find_all('tr', {"class": "data-table-row"})
    for row in cprows: 
        name = row.find('td', {"class": "data-table-row-cell", "data-type": "name"}).text.strip()
        price_str = row.find('td', {"class": "data-table-row-cell", "data-type": "plain-text"})
        if price_str is None:
            cp.append({'name': name, 'price': None})
        else:
            cp.append({'name': name, 'price': float(price_str.text)})
            
    return cp

### Jump from one page to another
`dp = date_picker`

In [3]:
def xpath_equal_text(text):
    return "//*[text()='{}')]".format(text)
def xpath_contain_text(text):
    return "//*[contains(text(), '{}}')]".format(text)

In [5]:
def rand_time_sleep(low, high, maxval):
    """This function simulates normal browsing behavior by introducing random delays between page jumps"""
    i = uniform(0, 1)
    if i < 0.85:
        sleep_time = randint(low, high)
    else: 
        sleep_time = randint(high, maxval)
    print("sleep for {} secs".format(sleep_time))
    sleep(sleep_time)

In [7]:
def update_page_date(driver, target_date):
    dp = driver.find_element_by_id('datepicker')
    for _ in range(10):
        dp.send_keys(Keys.BACKSPACE)
    dp.send_keys(target_date.strftime('%m/%d/%Y'))
    for _ in range(2):
        dp.send_keys(Keys.ENTER)
    sleep(1)

In [8]:
def str_to_date(dtstr):
    try:
        return datetime.strptime(dtstr, '%m/%d/%Y')
    except:
        print('Wrong date format. Follow this date format: `01/01/2019`')
        return None

In [9]:
def day_delta (start_date, end_date):
    delta = end_date - start_date
    return delta.days + 1

In [10]:
def check_cp_timezone(driver, location):
    """This function ensures the page's timezone is consistent with the `location` argument"""
    driver.implicitly_wait(2)
    dropdown = driver.find_element_by_class_name('data-table__time').find_element_by_tag_name('select')
    dropdown.click()
    drop_options = driver.find_elements_by_tag_name('option')
    for option in drop_options:
        if location in option.text:
            option.click()
            sleep(2)
            return

In [19]:
def bloomberg_cp_scraper(start_date, end_date):
    cp_hist = {}
    url = 'https://www.bloomberg.com/markets/currencies/fx-fixings'
    driver = webdriver.Chrome(executable_path='../_driver/chromedriver')
    driver.get(url)
    check_cp_timezone(driver, "London")
    try:
        start = str_to_date(start_date)
        end = str_to_date(end_date)
        days = day_delta(start, end)
        for decr_day in range(days):
            d = end - timedelta(days=decr_day)
            update_page_date(driver, d)
            rand_time_sleep(1, 3, 20)
            html_source = driver.page_source
            bs = BeautifulSoup(html_source, 'html.parser')
            cp = extract_currency_pairs(bs)
            print("Date: {}, {}".format(d, cp[0]))
            cp_hist[d] = cp
    except:
        return cp_hist
        
    driver.quit()
    return cp_hist

In [22]:
cp18 = bloomberg_cp_scraper('01/01/2018', '12/31/2018')

sleep for 1 secs
Date: 2018-12-31 00:00:00, {'name': 'MXN-CAD', 'price': 0.0694}
sleep for 1 secs
Date: 2018-12-30 00:00:00, {'name': 'MXN-CAD', 'price': None}
sleep for 6 secs
Date: 2018-12-29 00:00:00, {'name': 'MXN-CAD', 'price': None}
sleep for 2 secs
Date: 2018-12-28 00:00:00, {'name': 'MXN-CAD', 'price': 0.0695}
sleep for 1 secs
Date: 2018-12-27 00:00:00, {'name': 'MXN-CAD', 'price': 0.0692}
sleep for 1 secs
Date: 2018-12-26 00:00:00, {'name': 'MXN-CAD', 'price': 0.0685}
sleep for 2 secs
Date: 2018-12-25 00:00:00, {'name': 'MXN-CAD', 'price': None}
sleep for 3 secs
Date: 2018-12-24 00:00:00, {'name': 'MXN-CAD', 'price': 0.0685}
sleep for 3 secs
Date: 2018-12-23 00:00:00, {'name': 'MXN-CAD', 'price': None}
sleep for 1 secs
Date: 2018-12-22 00:00:00, {'name': 'MXN-CAD', 'price': None}
sleep for 1 secs
Date: 2018-12-21 00:00:00, {'name': 'MXN-CAD', 'price': 0.0681}
sleep for 2 secs
Date: 2018-12-20 00:00:00, {'name': 'MXN-CAD', 'price': 0.0678}
sleep for 1 secs
Date: 2018-12-19 00:0

sleep for 1 secs
Date: 2018-09-20 00:00:00, {'name': 'MXN-CAD', 'price': 0.0684}
sleep for 1 secs
Date: 2018-09-19 00:00:00, {'name': 'MXN-CAD', 'price': 0.0691}
sleep for 3 secs
Date: 2018-09-18 00:00:00, {'name': 'MXN-CAD', 'price': 0.0692}
sleep for 2 secs
Date: 2018-09-17 00:00:00, {'name': 'MXN-CAD', 'price': 0.0691}
sleep for 3 secs
Date: 2018-09-16 00:00:00, {'name': 'MXN-CAD', 'price': None}


In [25]:
pd.DataFrame.from_dict(cp18)

,2018-12-31 00:00:00,2018-12-30 00:00:00,2018-12-29 00:00:00,2018-12-28 00:00:00,2018-12-27 00:00:00,2018-12-26 00:00:00,2018-12-25 00:00:00,2018-12-24 00:00:00,2018-12-23 00:00:00,2018-12-22 00:00:00,...,2018-09-25 00:00:00,2018-09-24 00:00:00,2018-09-23 00:00:00,2018-09-22 00:00:00,2018-09-21 00:00:00,2018-09-20 00:00:00,2018-09-19 00:00:00,2018-09-18 00:00:00,2018-09-17 00:00:00,2018-09-16 00:00:00
0,"{'name': 'MXN-CAD', 'price': 0.0694}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': 0.0695}","{'name': 'MXN-CAD', 'price': 0.0692}","{'name': 'MXN-CAD', 'price': 0.0685}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': 0.0685}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': None}",...,"{'name': 'MXN-CAD', 'price': 0.0682}","{'name': 'MXN-CAD', 'price': 0.0686}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': None}","{'name': 'MXN-CAD', 'price': 0.0685}","{'name': 'MXN-CAD', 'price': 0.0684}","{'name': 'MXN-CAD', 'price': 0.0691}","{'name': 'MXN-CAD', 'price': 0.0692}","{'name': 'MXN-CAD', 'price': 0.0691}","{'name': 'MXN-CAD', 'price': None}"
1,"{'name': 'GBP-CAD', 'price': 1.7406}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': 1.7291}","{'name': 'GBP-CAD', 'price': 1.7234}","{'name': 'GBP-CAD', 'price': 1.7245}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': 1.7271}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': None}",...,"{'name': 'GBP-CAD', 'price': 1.7043}","{'name': 'GBP-CAD', 'price': 1.699}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': 1.6913}","{'name': 'GBP-CAD', 'price': 1.7104}","{'name': 'GBP-CAD', 'price': 1.7049}","{'name': 'GBP-CAD', 'price': 1.7089}","{'name': 'GBP-CAD', 'price': 1.7129}","{'name': 'GBP-CAD', 'price': None}"
2,"{'name': 'USD-DKK', 'price': 6.52775}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': 6.5293}","{'name': 'USD-DKK', 'price': 6.5458}","{'name': 'USD-DKK', 'price': 6.54495}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': 6.54215}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': None}",...,"{'name': 'USD-DKK', 'price': 6.3307}","{'name': 'USD-DKK', 'price': 6.3279}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': None}","{'name': 'USD-DKK', 'price': 6.34195}","{'name': 'USD-DKK', 'price': 6.34845}","{'name': 'USD-DKK', 'price': 6.3924}","{'name': 'USD-DKK', 'price': 6.37645}","{'name': 'USD-DKK', 'price': 6.3825}","{'name': 'USD-DKK', 'price': None}"
3,"{'name': 'XAU-USD', 'price': 1280.59}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': 1278.19}","{'name': 'XAU-USD', 'price': 1274.63}","{'name': 'XAU-USD', 'price': 1278.52}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': 1265.67}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': None}",...,"{'name': 'XAU-USD', 'price': 1202.04}","{'name': 'XAU-USD', 'price': 1203.55}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': None}","{'name': 'XAU-USD', 'price': 1199.99}","{'name': 'XAU-USD', 'price': 1204.77}","{'name': 'XAU-USD', 'price': 1203.43}","{'name': 'XAU-USD', 'price': 1199.64}","{'name': 'XAU-USD', 'price': 1203.73}","{'name': 'XAU-USD', 'price': None}"
4,"{'name': 'USD-MXN', 'price': 19.6911}","{'name': 'USD-MXN', 'price': None}","{'name': 'USD-MXN', 'price': None}","{'name': 'USD-MXN', 'price': 19.6214}","{'name': 'USD-MXN', 'price': 19.6959}","{'name': 'USD-MXN', 'price': 19.8673}","{'name': 'USD-MXN', 'price': None}","{'name': 'USD-MXN', 'price': 19.8302}","{'name': 'USD-MXN', 'price': None}","{'name': 'USD-MXN', 'price': None}",...,"{'name': 'USD-MXN', 'price': 18.9979}","{'name': 'USD-MXN', 'price': 18.8433}","{'name': 'USD-MXN', 'price

In [29]:
cp18_123qr = bloomberg_cp_scraper('01/01/2018', '01/01/2018')

sleep for 2 secs
Date: 2018-01-01 00:00:00, {'name': 'MXN-CAD', 'price': None}


In [34]:
print(cp18_123qr)

{datetime.datetime(2018, 1, 1, 0, 0): [{'name': 'MXN-CAD', 'price': None}, {'name': 'GBP-CAD', 'price': None}, {'name': 'USD-DKK', 'price': None}, {'name': 'XAU-USD', 'price': None}, {'name': 'USD-MXN', 'price': None}, {'name': 'THB-CAD', 'price': None}, {'name': 'BRL-CAD', 'price': None}, {'name': 'ZAR-CAD', 'price': None}, {'name': 'USD-JPY', 'price': None}, {'name': 'GBP-JPY', 'price': None}, {'name': 'USD-CNH', 'price': None}, {'name': 'EUR-PHP', 'price': None}, {'name': 'USD-ILS', 'price': None}, {'name': 'EUR-KRW', 'price': None}, {'name': 'EUR-NZD', 'price': None}, {'name': 'PEN-CAD', 'price': None}, {'name': 'USD-SGD', 'price': None}, {'name': 'EUR-INR', 'price': None}, {'name': 'EUR-ZAR', 'price': None}, {'name': 'AUD-JPY', 'price': None}, {'name': 'EUR-CZK', 'price': None}, {'name': 'EUR-SEK', 'price': None}, {'name': 'USD-ZAR', 'price': None}, {'name': 'EUR-PLN', 'price': None}, {'name': 'USD-THB', 'price': None}, {'name': 'USD-CHF', 'price': None}, {'name': 'EUR-BRL', 'pric

In [63]:
cp1819 = {}
cp1819.update(cp_hist_1qr2qr_2018)
cp1819.update(cp_hist_3qr_2018)
cp1819.update(cp_hist)
print(cp1819)

{datetime.datetime(2018, 5, 31, 0, 0): [{'name': 'MXN-CAD', 'price': 0.0648}, {'name': 'GBP-CAD', 'price': 1.7265}, {'name': 'USD-DKK', 'price': 6.37575}, {'name': 'XAU-USD', 'price': 1305.25}, {'name': 'USD-MXN', 'price': 20.0036}, {'name': 'THB-CAD', 'price': 4.0459}, {'name': 'BRL-CAD', 'price': 0.3482}, {'name': 'ZAR-CAD', 'price': 0.1025}, {'name': 'USD-JPY', 'price': 108.575}, {'name': 'GBP-JPY', 'price': 144.535}, {'name': 'USD-CNH', 'price': 6.4062}, {'name': 'EUR-PHP', 'price': 61.3313}, {'name': 'USD-ILS', 'price': 3.5649}, {'name': 'EUR-KRW', 'price': 1256.99}, {'name': 'EUR-NZD', 'price': 1.6635}, {'name': 'PEN-CAD', 'price': 0.396}, {'name': 'USD-SGD', 'price': 1.3381}, {'name': 'EUR-INR', 'price': 78.7745}, {'name': 'EUR-ZAR', 'price': 14.7701}, {'name': 'AUD-JPY', 'price': 82.175}, {'name': 'EUR-CZK', 'price': 25.812}, {'name': 'EUR-SEK', 'price': 10.3033}, {'name': 'USD-ZAR', 'price': 12.6527}, {'name': 'EUR-PLN', 'price': 4.3164}, {'name': 'USD-THB', 'price': 32.057}, 

### Data Processing
Table pivoting and column naming for output to excel. 

In [35]:
def from_dict_to_df(d):
    df = pd.DataFrame.from_dict(d, orient='index')
    df.sort_index()
    
    cols = []
    for column in df:
        cols.append(df[column][str_to_date('01/01/2018')]['name'])
    df.columns = cols
    
    for i,j in df.iterrows():
        for col in df.columns:
            df[col][i] = df[col][i]['price']
            
    return df

In [36]:
df_first_day = from_dict_to_df(cp18_123qr)


In [64]:
df1819 = pd.DataFrame.from_dict(cp1819, orient='index')

In [66]:
df1819.sort_index()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
2018-01-02 00:00:00.000000,"{'name': 'MXN-CAD', 'price': 0.0641}","{'name': 'GBP-CAD', 'price': 1.6986}","{'name': 'USD-DKK', 'price': 6.18135}","{'name': 'XAU-USD', 'price': 1312.48}","{'name': 'USD-MXN', 'price': 19.5182}","{'name': 'THB-CAD', 'price': 3.8528}","{'name': 'BRL-CAD', 'price': 0.3833}","{'name': 'ZAR-CAD', 'price': 0.1004}","{'name': 'USD-JPY', 'price': 112.145}","{'name': 'GBP-JPY', 'price': 152.275}",...,"{'name': 'USD-HKD', 'price': 7.8157}","{'name': 'TWD-CAD', 'price': 4.225}","{'name': 'EUR-USD', 'price': 1.20425}","{'name': 'EUR-SGD', 'price': 1.6008}","{'name': 'EUR-MXN', 'price': 23.5048}","{'name': 'USD-CAD', 'price': 1.25105}","{'name': 'EUR-GBP', 'price': 0.88691}","{'name': 'IDR-CAD', 'price': 0.0926}","{'name': 'SEK-CAD', 'price': 0.1529}","{'name': 'EUR-JPY', 'price': 135.06}"
2018-01-03 00:00:00.000000,"{'name': 'MXN-CAD', 'price': 0.0645}","{'name': 'GBP-CAD', 'price': 1.6939}","{'name': 'USD-DKK', 'price': 6.1926}","{'name': 'XAU-USD', 'price': 1316.46}","{'name': 'USD-MXN', 'price': 19.4351}","{'name': 'THB-CAD', 'price': 3.8839}","{'name': 'BRL-CAD', 'price': 0.3851}","{'name': 'ZAR-CAD', 'price': 0.1014}","{'name': 'USD-JPY', 'price': 112.4}","{'name': 'GBP-JPY', 'price': 151.897}",...,"{'name': 'USD-HKD', 'price': 7.8166}","{'name': 'TWD-CAD', 'price': 4.241}","{'name': 'EUR-USD', 'price': 1.2022}","{'name': 'EUR-SGD', 'price': 1.5982}","{'name': 'EUR-MXN', 'price': 23.3649}","{'name': 'USD-CAD', 'price': 1.2535}","{'name': 'EUR-GBP', 'price': 0.8896}","{'name': 'IDR-CAD', 'price': 0.093}","{'name': 'SEK-CAD', 'price': 0.1534}","{'name': 'EUR-JPY', 'price': 135.13}"
2018-01-04 00:00:00.000000,"{'name': 'MXN-CAD', 'price': 0.065}","{'name': 'GBP-CAD', 'price': 1.697}","{'name': 'USD-DKK', 'price': 6.1649}","{'name': 'XAU-USD', 'price': 1319.7}","{'name': 'USD-MXN', 'price': 19.251}","{'name': 'THB-CAD', 'price': 3.887}","{'name': 'BRL-CAD', 'price': 0.3882}","{'name': 'ZAR-CAD', 'price': 0.1018}","{'name': 'USD-JPY', 'price': 112.77}","{'name': 'GBP-JPY', 'price': 152.83}",...,"{'name': 'USD-HKD', 'price': 7.8172}","{'name': 'TWD-CAD', 'price': 4.2325}","{'name': 'EUR-USD', 'price': 1.2077}","{'name': 'EUR-SGD', 'price': 1.6044}","{'name': 'EUR-MXN', 'price': 23.2494}","{'name': 'USD-CAD', 'price': 1.2523}","{'name': 'EUR-GBP', 'price': 0.89116}","{'name': 'IDR-CAD', 'price': 0.0933}","{'name': 'SEK-CAD', 'price': 0.1539}","{'name': 'EUR-JPY', 'price': 136.2}"
2018-01-05 00:00:00.000000,"{'name': 'MXN-CAD', 'price': 0.0643}","{'name': 'GBP-CAD', 'price': 1.6811}","{'name': 'USD-DKK', 'price': 6.18785}","{'name': 'XAU-USD', 'price': 1318.83}","{'name': 'USD-MXN', 'price': 19.2936}","{'name': 'THB-CAD', 'price': 3.851}","{'name': 'BRL-CAD', 'price': 0.3833}","{'name': 'ZAR-CAD', 'price': 0.1004}","{'name': 'USD-JPY', 'price': 113.22}","{'name': 'GBP-JPY', 'price': 153.542}",...,"{'name': 'USD-HKD', 'price': 7.8194}","{'name': 'TWD-CAD', 'price': 4.2033}","{'name': 'EUR-USD', 'price': 1.20335}","{'name': 'EUR-SGD', 'price': 1.5972}","{'name': 'EUR-MXN', 'price': 23.2169}","{'name': 'USD-CAD', 'price': 1.2397}","{'name': 'EUR-GBP', 'price': 0.88736}","{'name': 'IDR-CAD', 'price': 0.0924}","{'name': 'SEK-CAD', 'price': 0.1522}","{'name': 'EUR-JPY', 'price': 136.25}"
2018-01-06 00:00:00.000000,"{'name': 'MXN-CAD', 'price': None}","{'name': 'GBP-CAD', 'price': None}","{'name': 'USD-DKK', 'price': None}","{'name': 'XAU-USD', 'price': None}","{'name': 'USD-MXN', 'price': None}","{'name': 'THB-CAD', 'price': None}","{'name': 'BRL-CAD', 'price': None}","{'name': 'ZAR-CAD', 'price': None}","{'name': 'USD-JPY', 'price': None}","{'name': 'GBP-JPY', 'price': None}",...,"{'name': 'USD-HKD', 'price': None}","{'name': 'TWD-CAD', 'price': None}","{'name': 'EUR-USD', 'price': None}","{'name': 'EUR-SGD', 'price': None}","{'name': 'EUR-MXN', 'price': None}","{'name': 'USD-CAD', 'price': None}","{'name': 'EUR-GBP', 'price': None}","{'name': 'IDR-CAD', 'price'

In [75]:
cols = []
for column in df1819:
    cols.append(df1819[column][str_to_date('01/02/2018')]['name'])
df1819.columns = cols

['MXN-CAD', 'GBP-CAD', 'USD-DKK', 'XAU-USD', 'USD-MXN', 'THB-CAD', 'BRL-CAD', 'ZAR-CAD', 'USD-JPY', 'GBP-JPY', 'USD-CNH', 'EUR-PHP', 'USD-ILS', 'EUR-KRW', 'EUR-NZD', 'PEN-CAD', 'USD-SGD', 'EUR-INR', 'EUR-ZAR', 'AUD-JPY', 'EUR-CZK', 'EUR-SEK', 'USD-ZAR', 'EUR-PLN', 'USD-THB', 'USD-CHF', 'EUR-BRL', 'EUR-DKK', 'EUR-CAD', 'USD-CZK', 'EUR-ILS', 'CHF-CAD', 'EUR-IDR', 'CAD-JPY', 'TRY-CAD', 'NOK-CAD', 'EUR-HKD', 'EUR-THB', 'EUR-MYR', 'EUR-TRY', 'SAR-CAD', 'USD-SEK', 'INR-CAD', 'MYR-CAD', 'USD-PLN', 'EUR-RON', 'GBP-USD', 'EUR-BGN', 'EUR-AUD', 'EUR-HRK', 'EUR-HUF', 'USD-HUF', 'NZD-USD', 'RUB-CAD', 'HKD-CAD', 'AUD-USD', 'USD-TRY', 'AUD-CAD', 'CNY-CAD', 'KRW-CAD', 'EUR-NOK', 'EUR-CHF', 'EUR-CNY', 'SGD-CAD', 'NZD-CAD', 'CHF-JPY', 'EUR-RUB', 'JPY-CAD', 'USD-HKD', 'TWD-CAD', 'EUR-USD', 'EUR-SGD', 'EUR-MXN', 'USD-CAD', 'EUR-GBP', 'IDR-CAD', 'SEK-CAD', 'EUR-JPY']


In [88]:
for i,j in df1819.iterrows():
    for col in df1819.columns:
        df1819[col][i] = df1819[col][i]['price']

In [95]:
df1819

,MXN-CAD,GBP-CAD,USD-DKK,XAU-USD,USD-MXN,THB-CAD,BRL-CAD,ZAR-CAD,USD-JPY,GBP-JPY,...,USD-HKD,TWD-CAD,EUR-USD,EUR-SGD,EUR-MXN,USD-CAD,EUR-GBP,IDR-CAD,SEK-CAD,EUR-JPY
2018-01-02 00:00:00.000000,0.0641,1.6986,6.18135,1312.48,19.5182,3.8528,0.3833,0.1004,112.145,152.275,...,7.8157,4.225,1.20425,1.6008,23.5048,1.25105,0.88691,0.0926,0.1529,135.06
2018-01-03 00:00:00.000000,0.0645,1.6939,6.1926,1316.46,19.4351,3.8839,0.3851,0.1014,112.4,151.897,...,7.8166,4.241,1.2022,1.5982,23.3649,1.2535,0.8896,0.093,0.1534,135.13
2018-01-04 00:00:00.000000,0.065,1.697,6.1649,1319.7,19.251,3.887,0.3882,0.1018,112.77,152.83,...,7.8172,4.2325,1.2077,1.6044,23.2494,1.2523,0.89116,0.0933,0.1539,136.2
2018-01-05 00:00:00.000000,0.0643,1.6811,6.18785,1318.83,19.2936,3.851,0.3833,0.1004,113.22,153.542,...,7.8194,4.2033,1.20335,1.5972,23.2169,1.2397,0.88736,0.0924,0.1522,136.25
2018-01-06 00:00:00.000000,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2018-01-07 00:00:00.000000,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2018-01-08 00:00:00.000000,0.0646,1.6857,6.21965,1319.54,19.2239,3.8536,0.3848,0.1001,113,153.409,...,7.8219,4.2047,1.1973,1.5953,23.0168,1.2417,0.88192,0.0925,0.1513,135.29
2018-01-09 00:00:00.000000,0.0646,1.6853,6.2418,1312.3,19.2889,3.8621,0.3838,0.1007,112.41,152.035,...,7.8201,4.2166,1.19305,1.5931,23.0126,1.2461,0.88211,0.0927,0.1512,134.11
2018-01-10 00:00:00.000000,0.0648,1.6888,6.2147,1319.41,19.2545,3.8874,0.3851,0.1003,111.34,150.654,...,7.8218,4.2165,1.1983,1.5979,23.0727,1.2481,0.8856,0.0929,0.1526,133.42
2018-01-11 00:00:00.000000,0.0648,1.6969,6.18075,1321.01,19.3356,3.921,0.3889,0.1009,111.395,150.795,...,7.8231,4.2334,1.20495,1.6021,23.2984,1.2536,0.89012,0.0936,0.1541,134.23


In [97]:
df1819.to_csv('bloomberg_fix_1819.csv')